### Solving Aiyagari (1994) model
#### 07/2019

#### Calibrating the Markov process


We need to calibrate the markov process so that:

\begin{equation}
\begin{bmatrix}
e & u
\end{bmatrix}
\begin{bmatrix}
\pi_{ee} & \pi_{eu} \\
\pi_{ue} & \pi_{uu}
\end{bmatrix} = \begin{bmatrix}
e & u
\end{bmatrix}
\end{equation}

with $e = 0.9 \text{ and } \pi_{ee} = 0.9$

We get that:
\begin{equation}
\begin{bmatrix}
\pi_{ee} & \pi_{eu} \\
\pi_{ue} & \pi_{uu}
\end{bmatrix} = \begin{bmatrix}
0.9 & 0.1 \\
0.9 & 0.1
\end{bmatrix}
\end{equation}

In [53]:
import numpy as np
from scipy import interpolate
import numba as nb

#Economic parameters
Π = np.array([[0.9,0.1], [0.9, 0.1]] )
Uss = 0.1
α = 0.36
β = 0.96
δ = 0.06



# Basic functions

# Return on capital
rf = lambda k,u: α*k**(α - 1) * (1 - u)** (1 - α )


# Return on labor
wf = lambda k,u: (1-α)*k**(α ) * (1 - u)** (- α )


#Aggregate labor supply
Lf = lambda u: 1 - u


#Utility
def u(c):
    if c > 0:
        return np.log(c)
    else:
        return -500000

# Interpolant to evaluate Value Function
def interpolant(x,Kgrid,Vp):
    t = interpolate.interp1d(Kgrid,Vp,fill_value="extrapolate")
    return t(x)


def Bellman(Kp, *args):
    R, W, Kgrid, Vp, Ε, j, l, Π = args
    
    Today = u(R* Kgrid[j] + W*Ε[l] + (1 - δ) * Kgrid[j] - Kp)
#     print("Value for Today", Today)
    exp= 0
    for i in range(Nϵ):
        exp +=  Π[l,i]*interpolant(Kp,Kgrid,Vp[:,i])  
    
    Value = Today + β * exp

    return - Value



def simulate(Kp, simuT, Π, Kstar, Kgrid):

    np.random.seed(seed=13)
    ϵ = np.random.binomial(1, Π[0,1], simuT)
    Ksimu = np.zeros([simuT])
    
    Ksimu[0] = Kstar*3

    for t in range(1,simuT):
        print("K simulated",Ksimu[t-1])
        Ksimu[t] = interpolant(Ksimu[t-1], Kgrid, Kp[:,ϵ[t]])
    
    Kstar_new = np.mean(Ksimu[500:])
    
    return Kstar_new
    


#### Compute the steady state equilibrium for K

In [54]:
from scipy.optimize import minimize
from matplotlib import pyplot as plt


# Computational parameters
N = 50
Nϵ = 2
epsi = 1
epsi2 = 1
tol = 1e-6
maxiter = 500
simuT = 10000


# Make grid
Klow   = 1
Khigh  = 10
Kgrid = np.linspace(Klow, Khigh, N)

#Steady State capital
Kstar = ( (1/β - 1+δ) /(α*(1-Uss)**(1-α))) ** (1 / (α - 1))
Ustar = Uss
Ε = np.array([1,0])


iter  = 0
iter2 = 0
Vp=Kstar*np.ones([N,Nϵ])
Kp=Klow*np.ones([N,Nϵ])
Vp_new=np.zeros([N,Nϵ])
Kp_new=np.zeros([N,Nϵ])



while (epsi2 > tol) & (iter2 < maxiter):
    epsi = 1
    iter = 0
    # Dynamic Programming Step
    ##########################
    R = rf(Kstar, Ustar)
    print("Return ", R)
    W = wf(Kstar, Ustar)
    print("Wage ", W)
    
    while (epsi > tol) & (iter < maxiter):
    
        for j in range(N):

            for l in range(Nϵ):

                params = R, W, Kgrid, Vp, Ε, j, l, Π
                solve = minimize(Bellman,Kp[j,l], args=(params))
                Kp_new[j,l] = solve.x
                Vp_new[j,l] = - solve.fun
                

        epsi = ((Vp_new - Vp) ** 2).sum()
        print("Error term for the Value Function: ", epsi)
        #Update
        Kp=Kp_new*1.
        Vp = Vp_new*1.
        iter=iter+1
        
    print("Policy Function",Kp_new)
    plt.plot(Kgrid, Kp_new[:,0])
    plt.plot(Kgrid, Kp_new[:,1])
    plt.show()
    print("Value Function",Vp_new)
    plt.plot(Kgrid, Vp_new[:,0])
    plt.plot(Kgrid, Vp_new[:,1])
    plt.show()
    
    # Simulation step
    #################
    Kstar_new = simulate(Kp, simuT, Π, Kstar, Kgrid)
    print("New value for Kstar", Kstar_new)
    epsi2 = (Kstar_new - Kstar) ** 2
    print("Error term for K*: ", epsi)
    Kstar = Kstar_new
        
    iter2=iter2+1


    

Return  0.10166666666666672
Wage  1.3033533628105798


ValueError: A value in x_new is below the interpolation range.

In [ ]:
print(Π[1,0])

In [ ]:
print(Kstar)

In [7]:
print(Ksimu)

[-1.86234901e+180              inf             -inf ...  0.00000000e+000
  0.00000000e+000  0.00000000e+000]
